Following Free Code Camps tutorial closely.

In [1]:
import ollama
import numpy as np
import torch

In [2]:
from langchain_ollama import OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings

model_name="sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(model = model_name,
                                   encode_kwargs={'normalize_embeddings': True})

/Users/akshaysundru2004/miniconda3/envs/LLM_Train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Lecture 1 Magnetic fields and magnetic force.pdf")
loader2 = PyPDFLoader("Lecture 4 Faraday’s law and induction.pdf")
docs = loader.load()
docs2 = loader2.load()

print(f"Total characters: {len(docs[2].page_content)}")
print(f"Total characters: {len(docs2[5].page_content)}")

Total characters: 1647
Total characters: 1765


In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_docs = docs + docs2

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 512,
    chunk_overlap = 64
)

#Make splits
splits = text_splitter.split_documents(all_docs)

In [33]:
print(docs[0].page_content[:500])
print(docs2[0].page_content[:500])

PHYS2003: Physics for Electrical Engineers, semester 1 2023 
Lecture 1 
1 
 
Lecture 1: Magnetic fields and magnetic force 
Learning outcomes 
Assessable 
• Calculate the force on moving charges in a magnetic field via the cross product. 
• Calculate the direction of resulting vectors from the cross product or, where 
appropriate, apply basic rules to determine the direction. 
• Calculate potential difference in a moving conductor. 
Understanding 
• Visualize and conceptually explain the motion 
PHYS2003: Physics for Electrical Engineers, semester 1 2023 
Lecture 4 
1 
 
Lecture 4: Faraday’s law and induction 
Learning outcomes 
Assessable 
• Use Faraday’s law and Lenz’s law to calculate induced current and emf. 
• Calculate energy transfer due to induction. 
Understanding 
• Visualize and conceptually explain induced currents and electric fields. 
1. Summary 
This lecture will cover Faraday’s law, Lenz’s law, and induction and energy transfer. 
Consider a loop of conducting wire conne

In [34]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,  # these are already LangChain `Document` objects
    embedding=embeddings,
    collection_name="circuit_docs",
    persist_directory="./chroma_db"
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

From RAG Tutorial this appears to end the embedding and indexing section, next video we will see retrieval

# Part3 - Retrieval

In this section code camp goes over the retrieval section

In [16]:
docsearch = retriever.invoke("Edwin H. Hall experiment")

In [17]:
print(docsearch)

[Document(id='46e28fff-164e-4429-83fa-ad31e20bd954', metadata={'total_pages': 7, 'page_label': '6', 'creationdate': "D:20230224061506Z00'00'", 'page': 5, 'moddate': "D:20230305080614Z00'00'", 'producer': 'macOS Version 12.5 (Build 21G72) Quartz PDFContext, AppendMode 1.1', 'source': 'Lecture 1 Magnetic fields and magnetic force.pdf', 'author': 'David Gozzard', 'creator': 'Microsoft® Word 2016'}, page_content='PHYS2003: Physics for Electrical Engineers, semester 1 2023 \nLecture 1 \n6 \n \n \nFigure 3: Simplified diagram of the J. J. Thomson experiment that discovered the electron. \n5.2. The Hall Effect \nIn a conductor, when there is no current through it, the electrons move randomly with no \nnet or average motion in any direction. When a current is passed through the conductor, \nthe electrons continue to move randomly, but with an average drift in the opposite \ndirection to the electric field causing the current. The drift speed 𝑣𝑑 is very smaller \ncompared with the speed of the 

In [35]:
docsearch2 = retriever.invoke("Explain Faraday's and Lenz's law")

In [36]:
print(docsearch2)

[Document(id='f487dde1-ea4b-4196-952a-8a1cd92eadd7', metadata={'creationdate': "D:20230224061645Z00'00'", 'page': 0, 'page_label': '1', 'creator': 'Microsoft® Word 2016', 'producer': 'macOS Version 12.5 (Build 21G72) Quartz PDFContext, AppendMode 1.1', 'moddate': "D:20230311062840Z00'00'", 'author': 'David Gozzard', 'total_pages': 10, 'source': 'Lecture 4 Faraday’s law and induction.pdf'}, page_content='PHYS2003: Physics for Electrical Engineers, semester 1 2023 \nLecture 4 \n1 \n \nLecture 4: Faraday’s law and induction \nLearning outcomes \nAssessable \n• Use Faraday’s law and Lenz’s law to calculate induced current and emf. \n• Calculate energy transfer due to induction. \nUnderstanding \n• Visualize and conceptually explain induced currents and electric fields. \n1. Summary \nThis lecture will cover Faraday’s law, Lenz’s law, and induction and energy transfer. \nConsider a loop of conducting wire connected to an ammeter. If we move a bar magnet \ntowards the loop, we will observe: 

# Part 4 - Generation

In [38]:
from langchain.prompts import PromptTemplate

template = """Answer the following question only using the following context:
{context}

If the answer is not contained in the context, respond with:
"I cannot answer this question because the necessary information was not found in the provided documents."

When answering, include the **source file name** and **slide/page number** if available.

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the following question only using the following context:\n{context}\n\nIf the answer is not contained in the context, respond with:\n"I cannot answer this question because the necessary information was not found in the provided documents."\n\nWhen answering, include the **source file name** and **slide/page number** if available.\n\nQuestion: {question}\n')

In [39]:
llm = OllamaLLM(model = "llama3.2")

In [21]:
chain = prompt | llm

In [40]:
print(chain.invoke({"context":docsearch, "question":"Describe an experiment used to find the Hall effect"}))

The Hall Effect is described as follows:

Imagine a copper strip with current flowing down the page. The electrons then drift up the page. If we apply a magnetic field pointing into the page, the force due to the magnetic field will deflect the electrons to the right-hand edge of the strip.

This caused a greater density of electrons at the right-hand edge of the strip, and so a separation of charges that produces an electric field within the strip. The force on the electrons due to the electric field (equivalently, their own mutual repulsion) will eventually cancel the force due to the magnetic field, resulting in a maximum value of the electric field.

The electric field results in a potential difference across the strip, which can be measured with a voltmeter.

Source file name: 'Lecture 1 Magnetic fields and magnetic force.pdf'
Slide/page number: Page 6


In [41]:
print(chain.invoke({"context":docsearch2, "question":"Explain Faraday's Law and how it relates to induction"}))

The source file is 'Lecture 4 Faraday’s law and induction.pdf' (both slide/page numbers are mentioned in the metadata).

Faraday's Law of Induction states that an electromotive force (emf) is induced in a closed loop of wire when the magnetic flux through the loop changes. The direction of the induced emf is such that it opposes the change in the magnetic flux.

The mathematical expression for Faraday's Law is:

ℰ = −dΦB/dt

where ℰ is the induced emf, ΦB is the magnetic flux, and t is time.

In an ideal coil of N turns, the equation becomes:

ℰ = −N(dΦB)/dt

This law applies to many situations where a change in the magnetic field through a loop induces an electromotive force.
